# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [3]:
def mask_test_edges_biased_negative_samples(adj, test_percent=10.):
    """
    Task 2: General Biased Negative Samples (B.N.S.) Directed Link
    Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :param test_percent: percentage of edges in test set
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements of adjacency matrix
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Retrieve all unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()
    edges_positive, _, _ = sparse_to_tuple(adj_tilde)

    # Number of positive (= to number of negative) test node pairs to sample
    num_test = int(np.floor(edges_positive.shape[0] / (100. / test_percent)))

    # Sampling of positive node pairs
    edges_idx = np.arange(edges_positive.shape[0])
    np.random.shuffle(edges_idx)
    test_edges_idx = edges_idx[:num_test]
    test_edges = edges_positive[test_edges_idx]

    # In this setting, the reverse node pairs constitute negative samples
    test_edges_false = np.fliplr(test_edges)

    # Get training incomplete adjacency matrix
    train_edges = np.delete(edges_positive, np.hstack([test_edges_idx]), axis = 0)
    #data = np.ones(train_edges.shape[0])
    #adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape = adj.shape)

    # Validation set: not implemented for Task 2
    # therefore, val_edges and val_edges_false are None
    return train_edges, val_edges, val_edges_false, test_edges, test_edges_false

In [4]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset in ['Wisconsin', 'Texas', 'Cornell']:
        from torch_geometric.datasets import WebKB
        from scipy import sparse

        graph_data = WebKB(root='directed_data/', name=dataset)
        data = graph_data[0]

        edges = data.edge_index
        nodes_number = data.x.shape[0]
        graph_np = np.zeros((nodes_number, nodes_number))
        for i in range(edges.shape[1]):
            graph_np[edges[0, i], edges[1, i]] = 1
    
        adj = sparse.csr_matrix(graph_np)
        
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [5]:
def get_graph(DATASET='cora'):
    adj_init, features = load_data(DATASET)

    train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    #train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    
    if DATASET == 'cora':
        nodes_number = 2708
    elif DATASET == 'citeseer':
        nodes_number = 3327
    elif DATASET == 'Wisconsin':
        nodes_number = 251
    elif DATASET == 'Texas':
        nodes_number = 183
    elif DATASET == 'Cornell':
        nodes_number = 183
        
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0], replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

# 定义模型

In [6]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [7]:
SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)

context_embedding_dim = 12
importance_embedding_dim = 48
beta = 5.0
ALPHA = 1.0
LR = 0.01
MOMENTUM = 0.9
EPOCH = 5
N = 29.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'Wisconsin'

In [8]:
all_best_auc = []
all_best_ap = []
for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()
    
    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    all_best_ap.append(auc_first_best_ap)
    all_best_auc.append(auc_first_best_auc)

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 199.6478
Epoch: 01, Loss: 199.2218
Epoch: 02, Loss: 198.4308
Epoch: 03, Loss: 197.3437
Epoch: 04, Loss: 196.0301
AP SCORE：  0.45677250951522597
AUC SCORE:  0.401875
Epoch: 00, Loss: 194.5551
Epoch: 01, Loss: 194.4879
Epoch: 02, Loss: 194.3622
Epoch: 03, Loss: 194.1873
Epoch: 04, Loss: 193.9723
AP SCORE：  0.4619272534781824
AUC SCORE:  0.41125
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 193.7258
Epoch: 01, Loss: 193.4249
Epoch: 02, Loss: 192.8604
Epoch: 03, Loss: 192.0719
Epoch: 04, Loss: 191.0988
AP SCORE：  0.47823042622105494
AUC SCORE:  0.43625
Epoch: 00, Loss: 189.9792
Epoch: 01, Loss: 189.9356
Epoch: 02, Loss: 189.8535
Epoch: 03, Loss: 189.7376
Epoch: 04, Loss: 189.5926
AP SCORE：  0.48623394282896737
AUC SCORE:  0.44875
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 189.4228
Epoch: 01, Loss: 189.1745
Epoch: 02, Loss: 188.7069
Epoch: 03, L

Epoch: 04, Loss: 145.8760
AP SCORE：  0.7024421296948201
AUC SCORE:  0.6937500000000001
Epoch: 00, Loss: 145.4161
Epoch: 01, Loss: 145.3767
Epoch: 02, Loss: 145.3021
Epoch: 03, Loss: 145.1963
Epoch: 04, Loss: 145.0631
AP SCORE：  0.7061920997321337
AUC SCORE:  0.698125
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 144.9061
Epoch: 01, Loss: 144.7941
Epoch: 02, Loss: 144.5821
Epoch: 03, Loss: 144.2812
Epoch: 04, Loss: 143.9018
AP SCORE：  0.71117778158566
AUC SCORE:  0.7025
Epoch: 00, Loss: 143.4536
Epoch: 01, Loss: 143.4146
Epoch: 02, Loss: 143.3408
Epoch: 03, Loss: 143.2361
Epoch: 04, Loss: 143.1046
AP SCORE：  0.7125233051026371
AUC SCORE:  0.705625
######################### 模块循环 ： 21 ##########################
Epoch: 00, Loss: 142.9496
Epoch: 01, Loss: 142.8405
Epoch: 02, Loss: 142.6339
Epoch: 03, Loss: 142.3405
Epoch: 04, Loss: 141.9705
AP SCORE：  0.7126966120325607
AUC SCORE:  0.708125
Epoch: 00, Loss: 141.5333
Epoch: 01, Loss: 141.4949
Epoch: 02, Loss

Epoch: 04, Loss: 114.7401
AP SCORE：  0.7906349595570108
AUC SCORE:  0.7849999999999999
Epoch: 00, Loss: 114.4602
Epoch: 01, Loss: 114.4443
Epoch: 02, Loss: 114.4141
Epoch: 03, Loss: 114.3714
Epoch: 04, Loss: 114.3179
AP SCORE：  0.7906349595570108
AUC SCORE:  0.7849999999999999
######################### 模块循环 ： 40 ##########################
Epoch: 00, Loss: 114.2550
Epoch: 01, Loss: 114.1874
Epoch: 02, Loss: 114.0592
Epoch: 03, Loss: 113.8771
Epoch: 04, Loss: 113.6472
AP SCORE：  0.7920719404329049
AUC SCORE:  0.78625
Epoch: 00, Loss: 113.3753
Epoch: 01, Loss: 113.3601
Epoch: 02, Loss: 113.3315
Epoch: 03, Loss: 113.2911
Epoch: 04, Loss: 113.2403
AP SCORE：  0.7935107791626544
AUC SCORE:  0.7875
######################### 模块循环 ： 41 ##########################
Epoch: 00, Loss: 113.1806
Epoch: 01, Loss: 113.1149
Epoch: 02, Loss: 112.9903
Epoch: 03, Loss: 112.8133
Epoch: 04, Loss: 112.5898
AP SCORE：  0.7936377376328241
AUC SCORE:  0.788125
Epoch: 00, Loss: 112.3255
Epoch: 01, Loss: 112.3111
Epoc

Epoch: 04, Loss: 98.6478
AP SCORE：  0.83343249787819
AUC SCORE:  0.825
######################### 模块循环 ： 59 ##########################
Epoch: 00, Loss: 98.6173
Epoch: 01, Loss: 98.5785
Epoch: 02, Loss: 98.5051
Epoch: 03, Loss: 98.4006
Epoch: 04, Loss: 98.2687
AP SCORE：  0.8346286063150197
AUC SCORE:  0.826875
Epoch: 00, Loss: 98.1125
Epoch: 01, Loss: 98.1051
Epoch: 02, Loss: 98.0911
Epoch: 03, Loss: 98.0713
Epoch: 04, Loss: 98.0464
AP SCORE：  0.8346286063150197
AUC SCORE:  0.826875
######################### 模块循环 ： 60 ##########################
Epoch: 00, Loss: 98.0171
Epoch: 01, Loss: 97.9792
Epoch: 02, Loss: 97.9074
Epoch: 03, Loss: 97.8054
Epoch: 04, Loss: 97.6765
AP SCORE：  0.8355140238444646
AUC SCORE:  0.82875
Epoch: 00, Loss: 97.5238
Epoch: 01, Loss: 97.5166
Epoch: 02, Loss: 97.5032
Epoch: 03, Loss: 97.4841
Epoch: 04, Loss: 97.4601
AP SCORE：  0.836003227623142
AUC SCORE:  0.829375
######################### 模块循环 ： 61 ##########################
Epoch: 00, Loss: 97.4319
Epoch: 01, Lo

Epoch: 01, Loss: 87.7469
Epoch: 02, Loss: 87.7388
Epoch: 03, Loss: 87.7273
Epoch: 04, Loss: 87.7129
AP SCORE：  0.8376678279565963
AUC SCORE:  0.8375
######################### 模块循环 ： 81 ##########################
Epoch: 00, Loss: 87.6959
Epoch: 01, Loss: 87.6699
Epoch: 02, Loss: 87.6205
Epoch: 03, Loss: 87.5503
Epoch: 04, Loss: 87.4616
AP SCORE：  0.8386065631344619
AUC SCORE:  0.838125
Epoch: 00, Loss: 87.3565
Epoch: 01, Loss: 87.3523
Epoch: 02, Loss: 87.3445
Epoch: 03, Loss: 87.3333
Epoch: 04, Loss: 87.3193
AP SCORE：  0.8390566212064712
AUC SCORE:  0.83875
######################### 模块循环 ： 82 ##########################
Epoch: 00, Loss: 87.3029
Epoch: 01, Loss: 87.2774
Epoch: 02, Loss: 87.2290
Epoch: 03, Loss: 87.1602
Epoch: 04, Loss: 87.0733
AP SCORE：  0.8375982878731378
AUC SCORE:  0.838125
Epoch: 00, Loss: 86.9704
Epoch: 01, Loss: 86.9663
Epoch: 02, Loss: 86.9587
Epoch: 03, Loss: 86.9478
Epoch: 04, Loss: 86.9343
AP SCORE：  0.8375982878731378
AUC SCORE:  0.838125
######################

...

Epoch: 04, Loss: 160.3118
AP SCORE：  0.7878790956727855
AUC SCORE:  0.809375
Epoch: 00, Loss: 159.6748
Epoch: 01, Loss: 159.6387
Epoch: 02, Loss: 159.5701
Epoch: 03, Loss: 159.4725
Epoch: 04, Loss: 159.3488
AP SCORE：  0.7907393747170405
AUC SCORE:  0.81375
######################### 模块循环 ： 10 ##########################
Epoch: 00, Loss: 159.2019
Epoch: 01, Loss: 159.0469
Epoch: 02, Loss: 158.7534
Epoch: 03, Loss: 158.3372
Epoch: 04, Loss: 157.8131
AP SCORE：  0.7904895814274376
AUC SCORE:  0.8150000000000001
Epoch: 00, Loss: 157.1952
Epoch: 01, Loss: 157.1568
Epoch: 02, Loss: 157.0840
Epoch: 03, Loss: 156.9803
Epoch: 04, Loss: 156.8494
AP SCORE：  0.7931268003294196
AUC SCORE:  0.8187500000000001
######################### 模块循环 ： 11 ##########################
Epoch: 00, Loss: 156.6943
Epoch: 01, Loss: 156.5437
Epoch: 02, Loss: 156.2587
Epoch: 03, Loss: 155.8545
Epoch: 04, Loss: 155.3456
AP SCORE：  0.7992454477402339
AUC SCORE:  0.82375
Epoch: 00, Loss: 154.7455
Epoch: 01, Loss: 154.7053
Epo

Epoch: 04, Loss: 119.4689
AP SCORE：  0.8402809302585812
AUC SCORE:  0.85375
Epoch: 00, Loss: 119.1079
Epoch: 01, Loss: 119.0796
Epoch: 02, Loss: 119.0262
Epoch: 03, Loss: 118.9508
Epoch: 04, Loss: 118.8561
AP SCORE：  0.8400480837757348
AUC SCORE:  0.85375
######################### 模块循环 ： 30 ##########################
Epoch: 00, Loss: 118.7451
Epoch: 01, Loss: 118.6575
Epoch: 02, Loss: 118.4914
Epoch: 03, Loss: 118.2558
Epoch: 04, Loss: 117.9588
AP SCORE：  0.838450224049722
AUC SCORE:  0.8518749999999999
Epoch: 00, Loss: 117.6082
Epoch: 01, Loss: 117.5809
Epoch: 02, Loss: 117.5292
Epoch: 03, Loss: 117.4563
Epoch: 04, Loss: 117.3649
AP SCORE：  0.8387577637322615
AUC SCORE:  0.8518749999999999
######################### 模块循环 ： 31 ##########################
Epoch: 00, Loss: 117.2577
Epoch: 01, Loss: 117.1725
Epoch: 02, Loss: 117.0113
Epoch: 03, Loss: 116.7824
Epoch: 04, Loss: 116.4939
AP SCORE：  0.8345426519618464
AUC SCORE:  0.85
Epoch: 00, Loss: 116.1533
Epoch: 01, Loss: 116.1269
Epoch: 0

Epoch: 01, Loss: 97.8238
Epoch: 02, Loss: 97.7309
Epoch: 03, Loss: 97.5989
Epoch: 04, Loss: 97.4322
AP SCORE：  0.8179830267096321
AUC SCORE:  0.8325
Epoch: 00, Loss: 97.2349
Epoch: 01, Loss: 97.2242
Epoch: 02, Loss: 97.2039
Epoch: 03, Loss: 97.1751
Epoch: 04, Loss: 97.1389
AP SCORE：  0.8179830267096321
AUC SCORE:  0.8325
######################### 模块循环 ： 50 ##########################
Epoch: 00, Loss: 97.0962
Epoch: 01, Loss: 97.0483
Epoch: 02, Loss: 96.9576
Epoch: 03, Loss: 96.8286
Epoch: 04, Loss: 96.6657
AP SCORE：  0.8187239273208888
AUC SCORE:  0.8318749999999999
Epoch: 00, Loss: 96.4728
Epoch: 01, Loss: 96.4623
Epoch: 02, Loss: 96.4424
Epoch: 03, Loss: 96.4141
Epoch: 04, Loss: 96.3785
AP SCORE：  0.8181056477509963
AUC SCORE:  0.83125
######################### 模块循环 ： 51 ##########################
Epoch: 00, Loss: 96.3364
Epoch: 01, Loss: 96.2896
Epoch: 02, Loss: 96.2009
Epoch: 03, Loss: 96.0747
Epoch: 04, Loss: 95.9154
AP SCORE：  0.8167514810843296
AUC SCORE:  0.830625
Epoch: 00, Los

Epoch: 04, Loss: 84.8975
AP SCORE：  0.7987283970668752
AUC SCORE:  0.815625
Epoch: 00, Loss: 84.7688
Epoch: 01, Loss: 84.7617
Epoch: 02, Loss: 84.7482
Epoch: 03, Loss: 84.7290
Epoch: 04, Loss: 84.7047
AP SCORE：  0.7987283970668752
AUC SCORE:  0.815625
######################### 模块循环 ： 70 ##########################
Epoch: 00, Loss: 84.6760
Epoch: 01, Loss: 84.6447
Epoch: 02, Loss: 84.5853
Epoch: 03, Loss: 84.5009
Epoch: 04, Loss: 84.3941
AP SCORE：  0.7965467020204355
AUC SCORE:  0.8143750000000001
Epoch: 00, Loss: 84.2674
Epoch: 01, Loss: 84.2603
Epoch: 02, Loss: 84.2469
Epoch: 03, Loss: 84.2277
Epoch: 04, Loss: 84.2035
AP SCORE：  0.7965467020204355
AUC SCORE:  0.8143750000000001
######################### 模块循环 ： 71 ##########################
Epoch: 00, Loss: 84.1749
Epoch: 01, Loss: 84.1441
Epoch: 02, Loss: 84.0857
Epoch: 03, Loss: 84.0026
Epoch: 04, Loss: 83.8975
AP SCORE：  0.7967822823149656
AUC SCORE:  0.8143750000000001
Epoch: 00, Loss: 83.7729
Epoch: 01, Loss: 83.7658
Epoch: 02, Los

Epoch: 01, Loss: 76.2771
Epoch: 02, Loss: 76.2342
Epoch: 03, Loss: 76.1731
Epoch: 04, Loss: 76.0958
AP SCORE：  0.7863410230361154
AUC SCORE:  0.808125
Epoch: 00, Loss: 76.0043
Epoch: 01, Loss: 75.9999
Epoch: 02, Loss: 75.9916
Epoch: 03, Loss: 75.9799
Epoch: 04, Loss: 75.9651
AP SCORE：  0.7863410230361154
AUC SCORE:  0.808125
######################### 模块循环 ： 90 ##########################
Epoch: 00, Loss: 75.9478
Epoch: 01, Loss: 75.9255
Epoch: 02, Loss: 75.8834
Epoch: 03, Loss: 75.8234
Epoch: 04, Loss: 75.7475
AP SCORE：  0.7856287723238647
AUC SCORE:  0.8075
Epoch: 00, Loss: 75.6576
Epoch: 01, Loss: 75.6534
Epoch: 02, Loss: 75.6454
Epoch: 03, Loss: 75.6341
Epoch: 04, Loss: 75.6199
AP SCORE：  0.7856287723238647
AUC SCORE:  0.8075
######################### 模块循环 ： 91 ##########################
Epoch: 00, Loss: 75.6032
Epoch: 01, Loss: 75.5814
Epoch: 02, Loss: 75.5400
Epoch: 03, Loss: 75.4810
Epoch: 04, Loss: 75.4064
AP SCORE：  0.7864204389905313
AUC SCORE:  0.808125
Epoch: 00, Loss: 75.318

In [9]:
print("MEAN AP : ", np.array(all_best_ap).mean())
print("STD AP : ", np.array(all_best_ap).std())
print("MEAN AUC : ", np.array(all_best_auc).mean())
print("STD AUC : ", np.array(all_best_auc).std())
"""
print("AUC FIRST BEST AUC: ", auc_first_best_auc)
print("AUC FIRST BEST AP: ", auc_first_best_ap)
print("AP FIRST BEST AUC: ", ap_first_best_auc)
print("AP FIRST BEST AP: ", ap_first_best_ap)
"""

MEAN AP :  0.8179435198700761
STD AP :  0.05280476626297034
MEAN AUC :  0.8341249999999999
STD AUC :  0.03773488935984841


'\nprint("AUC FIRST BEST AUC: ", auc_first_best_auc)\nprint("AUC FIRST BEST AP: ", auc_first_best_ap)\nprint("AP FIRST BEST AUC: ", ap_first_best_auc)\nprint("AP FIRST BEST AP: ", ap_first_best_ap)\n'